In [1]:
%config Completer.use_jedi = False                                       
import warnings                                                             
warnings.filterwarnings(action="ignore")
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
x = [ [2, 0], [4, 0], [6, 1], [8, 1], [10, 1], [12, 1], [14, 1] ] #[공부시간, 과외시간], 7행 2열
print(x)
print(type(x))

xData = np.array(x)
print(xData)
print(type(xData))

[[2, 0], [4, 0], [6, 1], [8, 1], [10, 1], [12, 1], [14, 1]]
<class 'list'>
[[ 2  0]
 [ 4  0]
 [ 6  1]
 [ 8  1]
 [10  1]
 [12  1]
 [14  1]]
<class 'numpy.ndarray'>


In [3]:
y = [0, 0, 0, 1, 1, 1, 1] #합격여부, 실제값, 1행 7열
print(y)
print(type(y))

yData = np.array(y)
print(yData)
print(type(yData))

#reshape() : numpy에서 데이터는 그대로 유지한 채 배열의 형태(차원;shape)를 변경한다.
#1행 7열 numpy 배열 -> 7행 1열 numpy 배열
yData = np.array(y).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


***
xData와 yData를 저장할 placeholder를 만든다. 
***

In [4]:
#placeholder에 numpy 배열을 대입하는 경우 shape 속성을 이용해서 대입될 numpy 배열의 차원(shape)을 지정해야한다.
#[None, 2] -> placeholder에 대입될 numpy 배열의 행의 갯수는 몇개라도 상관없고, 열의 갯수는 무조건 2개라는 의미이다.
# row는 속성에 맞는 데이터수(자료수)가 변할 수 있기때문에 None, column은 속성의 갯수는 삽입되는 데이터의 속성 열의 갯수와 같이 2를 지정
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

***
가중치(기울기)와 y절편(b)을 임의의 값으로 정한다.
***

In [5]:
a = tf.Variable(tf.random_uniform([2, 1], dtype=tf.float32)) # 행렬의 내적(행렬곱)을 이용하기 위해 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32)) 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')
print('*' * 100)
print(' a1 -> %f,\n a2-> %f,\n b: -> %f' % (sess.run(a)[0], sess.run(a)[1], sess.run(b)))
print('*' * 100)
print(' a1 -> {:f},\n a2-> {:f},\n b: -> {:f}'.format (sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))

a
[[0.86680484]
 [0.04889953]]
b
[0.7419169]
****************************************************************************************************
 a1 -> 0.866805,
 a2-> 0.048900,
 b: -> 0.741917
****************************************************************************************************
 a1 -> 0.866805,
 a2-> 0.048900,
 b: -> 0.741917


***
시그모이드 방정식, 오차함수, 경사 하강법
***

In [6]:
#Y = 1 / (1 + np.e ** -((a1 * x1) + (a2 * x2) + b))
#sigmoid() 메소드로 tensorflow에서 시그모이드 방정식을 계산한다.
#matmul() 메소드로 tensorflow에서 행렬의 내적을 계산한다.
y = tf.sigmoid( tf.matmul(X, a) + b ) # 시그모이드 방정식, 예측치

#loss =  - tf.reduce_mean( 실측치 * tf.log(예측치)) + ( ( 1 - 실측치 ) * tf.log(1 - 예측치) ))
loss =  - tf.reduce_mean(   (   Y   * tf.log(y)) +
                         (( 1 - Y ) * tf.log(1 - y))
                        ) # 오차함수

gradient_descent = tf.train.GradientDescentOptimizer(0.01).minimize(loss) #경사 하강법

***
sigmoid() 메소드를 실행한 결과로 에측값(0 또는 1; 예측치)을 계산한다.
***

In [7]:
#sigmoid() 메소드의 실행 결과가 0.5이상이면 1을 0.5미만이면 0을 리턴시킨다.
sess = tf.Session()

#cast(data, dtype) 메소드는 tensorflow에서 데이터 형변환을 한다.
expect = tf.cast(tf.constant([1.9, 4.3]), dtype=tf.int32)
print(sess.run(expect))

#cast() 메소드는 캐스팅할 데이터가 boolean 타입일 경우 True는 1로, False는 0으로 형변환 한다.
expect = tf.cast(tf.constant([True, False]), dtype=tf.float32)
print(sess.run(expect))
expect = tf.cast([0.5 >= 0.5, 0.5 < 0.5], dtype=tf.float32)
print(sess.run(expect))

[1 4]
[1. 0.]
[1. 0.]


In [8]:
# 시그모이드 함수의 실행 결과(예측치; y)로 최종 예측치를 계산한다.
expect = tf.cast( (y >= 0.5), dtype=tf.float32) #최종 예측치

***
최종 예측값(expect;예측치)와 실제값(실측치;Y)이 일치하는 정도(정확도,accuracy)를 계산한다.
***

In [9]:
sess = tf.Session()
# equal() 메소드로 tensorflow에서 두 인수로 지정한 두 값이 같으면 True, 다르면 False를 리턴한다.
print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 11)))
# 정확도는 equal() 메소드로 예측치와 실측치가 같은가 비교후 True 또는 False를 
# cast() 메소드로 1 또는 0으로 캐스팅하고, 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(expect, Y), dtype=tf.float32))

True False


***
학습 시킨다.
***

In [29]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(10001):
    # run() 메소드로 여러개를 한 번에 실행하려면 run() 메소드로 실행할 목록을 리스트로 묶어서 전달한다.
    #run([실행할 내용, ...], feed_dict={플레이스홀더이름 : 플레이스홀더에저장할데이터, .....})
    _, loss_, a_, b_, expect_, accuracy_ = sess.run([gradient_descent, loss, a, b, expect, accuracy],
                                                                    feed_dict={X : xData, Y : yData})
    if epoch % 300 == 0:
        #print(epoch, loss_, a_[0], a_[1], b_, accuracy_)
        print('epoch -> {:5d}, loss -> {:7.4f}, a1 -> {:7.4f}, a2 -> {:7.4f}, b -> {:7.4f}, accuracy -> {:7.4f}'
               .format( epoch, loss_, a_[0][0], a_[1][0], b_[0], accuracy_ ))
# ---- 학습 종료

print('실측치\n', yData, sep='')
print('최종 예상치\n', expect_, sep='')
print('정확도 -> {:6.2%}'.format( accuracy_ ), sep='')

epoch ->     0, loss ->  1.3803, a1 ->  0.7420, a2 ->  0.2168, b ->  0.0279, accuracy ->  0.5714
epoch ->   300, loss ->  0.4742, a1 ->  0.1739, a2 ->  0.1769, b -> -0.4999, accuracy ->  0.7143
epoch ->   600, loss ->  0.4216, a1 ->  0.2180, a2 ->  0.1877, b -> -0.8941, accuracy ->  0.8571
epoch ->   900, loss ->  0.3804, a1 ->  0.2584, a2 ->  0.1928, b -> -1.2431, accuracy ->  0.8571
epoch ->  1200, loss ->  0.3475, a1 ->  0.2954, a2 ->  0.1933, b -> -1.5545, accuracy ->  0.8571
epoch ->  1500, loss ->  0.3209, a1 ->  0.3297, a2 ->  0.1898, b -> -1.8348, accuracy ->  0.8571
epoch ->  1800, loss ->  0.2990, a1 ->  0.3615, a2 ->  0.1831, b -> -2.0891, accuracy ->  0.8571
epoch ->  2100, loss ->  0.2806, a1 ->  0.3912, a2 ->  0.1740, b -> -2.3216, accuracy ->  0.8571
epoch ->  2400, loss ->  0.2650, a1 ->  0.4192, a2 ->  0.1627, b -> -2.5358, accuracy ->  0.8571
epoch ->  2700, loss ->  0.2516, a1 ->  0.4456, a2 ->  0.1499, b -> -2.7344, accuracy ->  0.8571
epoch ->  3000, loss ->  0.239

In [31]:
# d = np.array([[5, 8]])
d = np.array([5, 9]).reshape(1, 2)
print('합격 'if sess.run(expect, feed_dict={X : d})[0][0] == 1.0 else '불합격')

불합격
